SOI1010_Machine_Learning_II___Assignment_1_2025

In [ ]:
import torch
from torchvision import datasets

trainset = datasets.MNIST(root='./data',train=True, download=True)

testset = datasets.MNIST(root='./data',train=False,download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 487kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.80MB/s]


In [ ]:
torch.manual_seed(0)
val_ratio = 0.1
train_size = len(trainset)
indices = torch.randperm(train_size)
split_idx = (int)(train_size * val_ratio)

train_idx = indices[split_idx:]
val_idx = indices[:split_idx]

train_data = trainset.data[train_idx].float()/255.
train_labels = trainset.targets[train_idx]
val_data = trainset.data[val_idx].float()/255.
val_labels = trainset.targets[val_idx]
test_data = testset.data.float()/255.
test_labels = testset.targets

In [ ]:
#Problem (a)
tmp_idx = 42 # 42 in the val's range.
tmp_image = val_data[tmp_idx]
tmp_labels = val_labels[tmp_idx]
pair_list = [] # (거리, 라벨) 저장.
for i in range(len(train_data)):
  now_image = train_data[i]
  now_labels = train_labels[i]
  distance = torch.sum((tmp_image-now_image)**2) #sqrt는 단조증가 함수라서 씌우지 않는다.
  pair_list.append((distance.item(),now_labels.item()))

sorted_list = sorted(pair_list, key=lambda pair:pair[0])

print("result \n")
print(sorted_list[:5])

def give_label(info: list):
  counting = {}
  for distance, label in info:
    if distance == 0:
      return label
    counting[label] = counting.get(label,0) + 1

  maximum = max(counting.values())

  nominate = []

  for label,count in counting.items():
    if count == maximum:
      nominate.append(label)

  if len(nominate) == 1:
    return nominate[0]

  else:
    check_winner = {}
    for distance, label in info:
      if label in nominate:
        check_winner[label] = check_winner.get(label,0) + 1/distance

    return max(check_winner, key=check_winner.get)
print(give_label(sorted_list[:5]))

result 

[(35.71235656738281, 0), (38.872032165527344, 0), (40.079063415527344, 0), (40.36872100830078, 0), (40.44904327392578, 0)]
0


In [ ]:
#Problem(b)
image_for_test = val_data[42]
print(image_for_test.shape)
plus_dimension = image_for_test.unsqueeze(0)
print(plus_dimension.shape)

print(train_data.shape)

distance = torch.sum((train_data-plus_dimension)**2,dim=1).sum(dim=1)
print(distance.shape)

small_distance, index = torch.topk(distance,k=5,largest=False)

print(small_distance)
print(train_labels[index])

torch.Size([28, 28])
torch.Size([1, 28, 28])
torch.Size([54000, 28, 28])
torch.Size([54000])
tensor([35.7124, 38.8720, 40.0791, 40.3687, 40.4490])
tensor([0, 0, 0, 0, 0])


In [ ]:
#Problem(c)
print(val_data.shape)
print(train_data.shape)

broad_val = val_data.unsqueeze(1)
broad_train = train_data.unsqueeze(0)
print(broad_val.shape)
print(broad_train.shape)

torch.Size([6000, 28, 28])
torch.Size([54000, 28, 28])
torch.Size([6000, 1, 28, 28])
torch.Size([1, 54000, 28, 28])


In [ ]:
#distance = torch.sum((broad_val-broad_train)**2,dim=2).sum(dim=2)

In [ ]:
#Problem(d)

In [ ]:
br_val = val_data.view(6000,-1)
br_tr = train_data.view(54000,-1)

total_distance = torch.cdist(br_val,br_tr,p=2)**2

In [ ]:
print(total_distance.shape)

torch.Size([6000, 54000])


In [ ]:
result = []
for row in total_distance:
  pair_list = []
  for i in range(0,54000):
    pair_list.append((row[i],train_labels[i]))
  now_sort = sorted(pair_list, key=lambda pair:pair[0])
  result.append(give_label(now_sort[:5]))

print(result)

KeyboardInterrupt: 

In [ ]:
distance_info, idx = torch.topk(total_distance,k=5,dim=1,largest=False)

labels = train_labels[idx]

print(labels.shape)

results, _ = torch.mode(labels,dim=1)

print(results.shape)
print(_.shape)

print(results)
print(val_labels[_])
print(_)

torch.Size([6000, 5])
torch.Size([6000])
torch.Size([6000])
tensor([9, 3, 5,  ..., 9, 9, 8])
tensor([7, 7, 7,  ..., 7, 7, 7])
tensor([4, 4, 4,  ..., 4, 4, 4])


In [ ]:
print(results[42])
print(give_label(sorted_list[:5]))

tensor(0)
0


In [ ]:
#Probelm(E): We can control val_ratio and number of k
#Problem(F):

In [ ]:
best_comb = []

for i in range(1,31):
  now_ratio = 0.01 * i

  split_idx = int(train_size * now_ratio)

  train_idx = indices[split_idx:]
  val_idx = indices[:split_idx]

  train_data = trainset.data[train_idx].float() / 255.
  train_labels = trainset.targets[train_idx]
  val_data = trainset.data[val_idx].float() / 255.
  val_labels = trainset.targets[val_idx]

  num_val = len(val_data)
  num_train = len(train_data)

  br_val = val_data.view(num_val, -1)
  br_tr = train_data.view(num_train,-1)

  total_distance = torch.cdist(br_val,br_tr,p=2)**2

  k_range = range(5,21)
  results_k = {}

  for k in k_range:
    _, idx = torch.topk(total_distance, k=k,dim=1, largest=False)
    labels = train_labels[idx]
    predictions, _ = torch.mode(labels,dim=1)
    accuracy = (predictions == val_labels).float().mean()

    results_k[k] = {'accuracy':accuracy.item()}

  best_k = max(results_k,key=lambda k: results_k[k]['accuracy'])
  best_comb.append({
    'ratio':now_ratio,'best_k':best_k,'accuracy':results_k[best_k]['accuracy']
})

  print(f"Ratio:{now_ratio}, k:{best_k}, Accuracy:{results_k[best_k]['accuracy']}")

Best = max(best_comb,key=lambda A: A['accuracy'])

final_ratio = Best['ratio']
final_k = Best['best_k']

print(f"Ratio:{final_ratio}, k:{final_k}")

Ratio:0.01, k:7, Accuracy:0.9783333539962769
Ratio:0.02, k:7, Accuracy:0.9733333587646484
Ratio:0.03, k:6, Accuracy:0.971666693687439
Ratio:0.04, k:5, Accuracy:0.9708333611488342
Ratio:0.05, k:5, Accuracy:0.971666693687439
Ratio:0.06, k:5, Accuracy:0.9708333611488342
Ratio:0.07, k:5, Accuracy:0.9719047546386719
Ratio:0.08, k:5, Accuracy:0.9735416769981384
Ratio:0.09, k:5, Accuracy:0.9729629755020142
Ratio:0.1, k:5, Accuracy:0.9721666574478149
Ratio:0.11, k:5, Accuracy:0.9727272987365723
Ratio:0.12, k:5, Accuracy:0.9715277552604675
Ratio:0.13, k:5, Accuracy:0.9715384840965271
Ratio:0.14, k:5, Accuracy:0.9725000262260437
Ratio:0.15, k:5, Accuracy:0.9725555777549744
Ratio:0.16, k:5, Accuracy:0.9729166626930237
Ratio:0.17, k:5, Accuracy:0.9728431105613708
Ratio:0.18, k:5, Accuracy:0.9734259247779846
Ratio:0.19, k:5, Accuracy:0.9727193117141724
Ratio:0.2, k:5, Accuracy:0.9722499847412109
Ratio:0.21, k:5, Accuracy:0.9719841480255127
Ratio:0.22, k:5, Accuracy:0.9717424511909485
Ratio:0.23, k:

In [ ]:
best_k = 7

train_data = trainset.data.float() / 255.
train_labels = trainset.targets

test_data = testset.data.float()/255.
test_labels = testset.targets

num_train = len(train_data)
num_test = len(test_data)
br_train = train_data.view(num_train,-1)
br_test = test_data.view(num_test,-1)

total_distance = torch.cdist(br_test,br_train,p=2)**2

_, idx = torch.topk(total_distance, k = best_k, dim = 1, largest=False)
labels = train_labels[idx]
predictions, _  = torch.mode(labels, dim=1)

accuracy = (predictions == test_labels).float().mean()

print(f"k:{best_k}, accuracy:{accuracy}")

k:7, accuracy:0.9693999886512756


In [ ]:
#Problem(G): Change Accuracy Algorithm.
#Instead of Accuracy, I will use F1 Score.

In [ ]:
best_combination = []

for i in range(1,31): # After accuracy exceeded 30, acc dropped below 97%, so only proceeded up to 30.
  now_ratio = 0.01 * i

  split_idx = int(train_size * now_ratio)

  train_idx = indices[split_idx:]
  val_idx = indices[:split_idx]

  train_data = trainset.data[train_idx].float() / 255.
  train_labels = trainset.targets[train_idx]
  val_data = trainset.data[val_idx].float() / 255.
  val_labels = trainset.targets[val_idx]

  num_val = len(val_data)
  num_train = len(train_data)

  br_val = val_data.view(num_val, -1)
  br_tr = train_data.view(num_train,-1)

  total_distance = torch.cdist(br_val,br_tr,p=2)**2

  k_range = range(5,21)

  results_k = {}
  epsilon = 1e-8

  for k in k_range:

    _, idx = torch.topk(total_distance,k=k,dim=1,largest=False)
    labels = train_labels[idx]
    predictions, _  = torch.mode(labels, dim=1)

    f1_scores = []

    for c in range(10):
      positive_number = c

      tp = ((predictions == positive_number)&(val_labels == positive_number)).sum().float()
      fp = ((predictions == positive_number)&(val_labels != positive_number)).sum().float()
      fn = ((predictions != positive_number)&(val_labels == positive_number)).sum().float()

      precision = tp / (tp + fp + epsilon)
      recall = tp / (tp + fn + epsilon)
      f1_score =  2 * (precision * recall) / (precision + recall + epsilon)

      f1_scores.append(f1_score)

    total_f1 = torch.stack(f1_scores).mean()

    results_k[k] = {'f1_score':total_f1.item()}

  best_k = max(results_k, key=lambda k :results_k[k]['f1_score'])
  best_combination.append({
      'ratio':now_ratio,
      'best_k':best_k,
      'f1_score':results_k[best_k]['f1_score']
  })

  print(f"Ratio :{now_ratio}, Best k: {best_k}, F1:{results_k[best_k]['f1_score']}")


Best = max(best_combination,key=lambda A: A['f1_score'])

print(f"Ratio:{Best['ratio']}, K:{Best['best_k']},F1:{Best['f1_score']}")

final_ratio = Best['ratio']
final_k = Best['best_k']

Ratio :0.01, Best k: 7, F1:0.9791160821914673
Ratio :0.02, Best k: 7, F1:0.9738765954971313
Ratio :0.03, Best k: 6, F1:0.9721206426620483
Ratio :0.04, Best k: 5, F1:0.9711148142814636
Ratio :0.05, Best k: 5, F1:0.9718723297119141
Ratio :0.06, Best k: 5, F1:0.9708722829818726
Ratio :0.07, Best k: 5, F1:0.9717548489570618
Ratio :0.08, Best k: 5, F1:0.9734271168708801
Ratio :0.09, Best k: 5, F1:0.9727876782417297
Ratio :0.1, Best k: 5, F1:0.9720419049263
Ratio :0.11, Best k: 5, F1:0.9726301431655884
Ratio :0.12, Best k: 5, F1:0.971340537071228
Ratio :0.13, Best k: 5, F1:0.971377968788147
Ratio :0.14, Best k: 5, F1:0.9723551869392395
Ratio :0.15, Best k: 5, F1:0.97236168384552
Ratio :0.16, Best k: 5, F1:0.9727203249931335
Ratio :0.17, Best k: 5, F1:0.9726755023002625
Ratio :0.18, Best k: 5, F1:0.9732446670532227
Ratio :0.19, Best k: 5, F1:0.972558319568634
Ratio :0.2, Best k: 5, F1:0.9721009135246277
Ratio :0.21, Best k: 5, F1:0.9718462228775024
Ratio :0.22, Best k: 5, F1:0.971596598625183

In [ ]:
#2. k = 7, F1 Score

best_k = 7
epsilon = 1e-8

train_data = trainset.data.float() / 255.
train_labels = trainset.targets

test_data = testset.data.float()/255.
test_labels = testset.targets

num_train = len(train_data)
num_test = len(test_data)
br_train = train_data.view(num_train,-1)
br_test = test_data.view(num_test,-1)

total_distance = torch.cdist(br_test,br_train,p=2)**2

_, idx = torch.topk(total_distance, k = best_k, dim = 1, largest=False)
labels = train_labels[idx]
predictions, _  = torch.mode(labels, dim=1)

f1_scores = []

for c in range(10):
  positive_number = c

  tp = ((predictions==positive_number)&(test_labels==positive_number)).sum().float()
  fp = ((predictions==positive_number)&(test_labels!=positive_number)).sum().float()
  fn = ((predictions!=positive_number)&(test_labels==positive_number)).sum().float()
  tn = ((predictions!=positive_number)&(test_labels!=positive_number)).sum().float()

  precision = tp/(tp+fp+epsilon)
  recall = tp / (tp+fn+epsilon)
  f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

  f1_scores.append(f1_score)

final_test_f1 = torch.stack(f1_scores).mean()

print(f"k:{best_k},F1:{final_test_f1}")

k:7,F1:0.9693803787231445


In [ ]:
#Additionally, Accuracy and F1 Score were measured with k=5, which recorded the next best performance after k=7.
best_k = 5
epsilon = 1e-8

train_data = trainset.data.float() / 255.
train_labels = trainset.targets

test_data = testset.data.float()/255.
test_labels = testset.targets

num_train = len(train_data)
num_test = len(test_data)
br_train = train_data.view(num_train,-1)
br_test = test_data.view(num_test,-1)

total_distance = torch.cdist(br_test,br_train,p=2)**2

_, idx = torch.topk(total_distance, k = best_k, dim = 1, largest=False)
labels = train_labels[idx]
predictions, _  = torch.mode(labels, dim=1)

f1_scores = []

for c in range(10):
  positive_number = c

  tp = ((predictions==positive_number)&(test_labels==positive_number)).sum().float()
  fp = ((predictions==positive_number)&(test_labels!=positive_number)).sum().float()
  fn = ((predictions!=positive_number)&(test_labels==positive_number)).sum().float()
  tn = ((predictions!=positive_number)&(test_labels!=positive_number)).sum().float()

  precision = tp/(tp+fp+epsilon)
  recall = tp / (tp+fn+epsilon)
  f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

  f1_scores.append(f1_score)

final_test_f1 = torch.stack(f1_scores).mean()
accuracy = (predictions==test_labels).float().mean()
print(f"k:{best_k},F1:{final_test_f1},Acc:{accuracy}")

k:5,F1:0.9687143564224243,Acc:0.9688000082969666


In [ ]:
'''
k:5,F1:0.9687143564224243,Acc:0.9688000082969666
k:7,F1:0.9693803787231445,accuracy:0.9693999886512756
'''

In [7]:
best_k = 7
epsilon = 1e-8

train_data = trainset.data.float() / 255.
train_labels = trainset.targets

test_data = testset.data.float()/255.
test_labels = testset.targets

num_train = len(train_data)
num_test = len(test_data)
br_train = train_data.view(num_train,-1)
br_test = test_data.view(num_test,-1)

total_distance = torch.cdist(br_test,br_train,p=2)**2

_, idx = torch.topk(total_distance, k = best_k, dim = 1, largest=False)
labels = train_labels[idx]
predictions, _  = torch.mode(labels, dim=1)

f1_scores = []

for c in range(10):
  positive_number = c

  tp = ((predictions==positive_number)&(test_labels==positive_number)).sum().float()
  fp = ((predictions==positive_number)&(test_labels!=positive_number)).sum().float()
  fn = ((predictions!=positive_number)&(test_labels==positive_number)).sum().float()
  tn = ((predictions!=positive_number)&(test_labels!=positive_number)).sum().float()

  precision = tp/(tp+fp+epsilon)
  recall = tp / (tp+fn+epsilon)
  f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

  f1_scores.append(f1_score)

final_test_f1 = torch.stack(f1_scores).mean()
accuracy = (predictions==test_labels).float().mean()

print("\n--- Each Label's Accuracy ---")

for i in range(10):
  correct_predictions = ((predictions == i) & (test_labels == i)).sum().item()
  total_instances = (test_labels == i).sum().item()
  if total_instances > 0:
    acc = correct_predictions / total_instances * 100
    print(f"{i}'s Accuracy: {acc:.2f}%")
  else:
    print(f"{i}'s Accuracy: N/A (no instances of this label in test set)")


--- Each Label's Accuracy ---
0's Accuracy: 99.39%
1's Accuracy: 99.82%
2's Accuracy: 95.74%
3's Accuracy: 96.63%
4's Accuracy: 96.23%
5's Accuracy: 97.09%
6's Accuracy: 98.54%
7's Accuracy: 96.21%
8's Accuracy: 94.05%
9's Accuracy: 95.44%


In [8]:
worst = (2, 7, 8, 9)

for i in worst:
    positive_number = i
    tp = ((predictions == positive_number) & (test_labels == positive_number)).sum().float()
    fp = ((predictions == positive_number) & (test_labels != positive_number)).sum().float()
    fn = ((predictions != positive_number) & (test_labels == positive_number)).sum().float()
    tn = ((predictions != positive_number) & (test_labels != positive_number)).sum().float()

    print(f"{i}'s score: Tp:{tp}, Tn:{tn}, Fp:{fp}, Fn:{fn}")

2's score: Tp:988.0, Tn:8951.0, Fp:17.0, Fn:44.0
7's score: Tp:989.0, Tn:8929.0, Fp:43.0, Fn:39.0
8's score: Tp:916.0, Tn:9013.0, Fp:13.0, Fn:58.0
9's score: Tp:963.0, Tn:8947.0, Fp:44.0, Fn:46.0
